__Install necessary packages__

In [1]:
# !pip install python-dotenv
# !pip install langchain langchain-core -U
# !pip install -qU langchain-openai


In [7]:
import os

from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [8]:
load_dotenv()

True

In [9]:
openai_api_key = os.getenv("OPENAI_API_KEY")

In [10]:
model = ChatOpenAI(model="gpt-3.5-turbo")

In [11]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        ('system', "Translate the following into {language}"),
        ('user', '{text}')
    ]
)
result = prompt_template.invoke(
    {
        'language': 'Persian',
        'text': 'Hello everyone!'
    }
)
result.to_messages()

[SystemMessage(content='Translate the following into Persian', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello everyone!', additional_kwargs={}, response_metadata={})]

In [12]:
parser = StrOutputParser()

In [13]:
chain = prompt_template | model | parser

In [14]:
chain.invoke(
    {
        'language': 'Persian',
        'text': 'hello everyone!'
    }
)

'سلام به همه\u200cی شما!'

# Few-shot learning

- for language model (for chat model it is different as we need a sequence of messages between AI and human in chat models)

In [15]:
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import FewShotPromptTemplate

In [16]:
example_prompt = PromptTemplate.from_template("Question: {question}\nAnswer:{answer}")
example_prompt

PromptTemplate(input_variables=['answer', 'question'], input_types={}, partial_variables={}, template='Question: {question}\nAnswer:{answer}')

In [17]:
examples = [
    {
        "question": "Who lived longer, Muhammad Ali or Alan Turing?",
        "answer": """
Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali
""",
    },
    {
        "question": "When was the founder of craigslist born?",
        "answer": """
Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952
""",
    },
    {
        "question": "Who was the maternal grandfather of George Washington?",
        "answer": """
Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washington?
Intermediate answer: The father of Mary Ball Washington was Joseph Ball.
So the final answer is: Joseph Ball
""",
    },
    {
        "question": "Are both the directors of Jaws and Casino Royale from the same country?",
        "answer": """
Are follow up questions needed here: Yes.
Follow up: Who is the director of Jaws?
Intermediate Answer: The director of Jaws is Steven Spielberg.
Follow up: Where is Steven Spielberg from?
Intermediate Answer: The United States.
Follow up: Who is the director of Casino Royale?
Intermediate Answer: The director of Casino Royale is Martin Campbell.
Follow up: Where is Martin Campbell from?
Intermediate Answer: New Zealand.
So the final answer is: No
""",
    },
]

In [18]:
print(example_prompt.invoke(examples[0]).to_string())

Question: Who lived longer, Muhammad Ali or Alan Turing?
Answer:
Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali



In [19]:
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix = "Question:{input}",
    input_variables = ['input'],
)
print(prompt.invoke({'input':"How are you?"}).to_string())

Question: Who lived longer, Muhammad Ali or Alan Turing?
Answer:
Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali


Question: When was the founder of craigslist born?
Answer:
Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952


Question: Who was the maternal grandfather of George Washington?
Answer:
Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the 

In [20]:
chain = prompt | model | parser

In [22]:
chain.invoke({'input':"How is the established of the Apple?"})

'Answer: \nAre follow up questions needed here: Yes.\nFollow up: Who co-founded Apple?\nIntermediate answer: Apple was co-founded by Steve Jobs, Steve Wozniak, and Ronald Wayne.\nFollow up: When was Apple established?\nIntermediate answer: Apple was established on April 1, 1976.\nSo the final answer is: Apple was established on April 1, 1976.'